### Using sourmash pangenome and compare to Roary
Roary uses the output from Prokka, then it extracts the coding regions in nucleotide space, then translates these internally using bioperl. It does a pangenome in protein space

#### Comparing Sourmash and Roary:
- Use sourmash on protein or predicted transcript sequences, not thw whole genome
- Use prokka output for this (ffn files (predicted transcripts, nucl), the faa files (predicted protein))
- Scale of 1? Later slide scale [1,10,50,100,1000] and k[21,31], protein_k [15,18,21]
- Tweak sourmash thresholds 

Using one bacterial species for now Lactobacillus amylovorus, because it has 90 genomes and that is a nice size. 


#### Results so far
Cannot get protein or nucleotide results to resemble what roary tells us. Maybe try skipmers (skipm1n3 and skipm2n3), when implemented


In [ ]:
# Try a couple settings, all at scale 10: 
# protein k12, k15 (dayhoff, k15,k18)
# nucleotide k21, k31
# translate k12,k15 (ffn)
# in results/test_params

# sourmash scripts pangenome merge for each 


In [ ]:
# How to get from a signature to a pangenome?
# sourmash sig cat protein_dayhoff/*.zip -o l_amylovorus.protein_dayhoff.zip
#sourmash sig cat transcript/*.zip -o l_amylovorus.transcript.zip
# sourmash sig cat transcript_dayhoff/*.zip -o l_amylovorus.transcript_dayhoff.zip
#sourmash sig cat transcript_tr/*.zip -o l_amylovorus.transcript_translate.zip

In [ ]:
# Results different from Roary, finds 9 core hashes, most is cloud
# try with prot sig, use k15, s10, try k18,k21 (18 worse than 15)
sourmash sig downsample \
../sourmash/protein/l_amylovorus.prot.zip -k 18 --scaled 10 \
--protein -o l_amylovorus.prot.18.10.zip

sourmash scripts pangenome_merge l_amylovorus.prot.18.10.zip -k 18 \
-o l_amylovorus.pang.prot.18.zip --scaled 10 --protein --no-dna

sourmash scripts pangenome_ranktable l_amylovorus.pang.prot.18.zip -o l_amylovorus.pang.prot.18.csv \
-k 18 --scaled 10 --protein --no-dna

mamba activate branchwater
python /group/ctbrowngrp2/amhorst/2025-pangenome/workflow/scripts/count_rank.py l_amylovorus.pang.prot.18.csv

In [ ]:
# These results are different from Roary, finds 1 core hash, no cloud.
# For DNA: use k21, s100 for now
# use dev env, cause am changing the pangenome code
mamba activate pangenomics_dev

# use genomes
sourmash sig downsample l_amylovorus.gtdb.zip -k 21 \
--scaled 100 -o l_amylovorus.21.100.zip

# create pangenome
sourmash scripts pangenome_merge ../sourmash/l_amylovorus.21.100.zip -k 21 \
-o l_amylovorus.pang.S2.zip --scaled 100

# ranktable
sourmash scripts pangenome_ranktable l_amylovorus.pang.S1.zip -o l_amylovorus.pang.S1.csv \
-k 21 --scaled 100

# print number of hashes for each rank:
mamba activate branchwater
python /group/ctbrowngrp2/amhorst/2025-pangenome/workflow/scripts/count_rank.py l_amylovorus.pang.S1.csv

In [ ]:
# Skipmers (don't work rn):
# Use whole genome? Protein? ffn?
mamba activate branchwater-skipmer

# Would need to do manysketch
echo name,genome_filename,protein_filename > manysketch.csv
for i in *.fna.gz
do
echo $i,$i,
done >> manysketch.csv


# activate skipmer env
mamba activate branchwater-skipmer

# Sketch skipmer (in folder MAGs)
sourmash scripts manysketch -p skipm2n3,k=21,k=24,k=27,scaled=50 --singleton \
-c 50 manysketch.csv -o ../skipmer/l_amylovorus.skipm2n3.zip

# also a 1n3
sourmash scripts manysketch -p skipm1n3,k=21,k=24,k=27,scaled=50 --singleton \
-c 50 manysketch.csv -o ../skipmer/l_amylovorus.skipm1n3.zip

# try pangenome merge and ranktable (m2n3, k21, s50)
sourmash scripts pangenome_merge l_amylovorus.skipm2n3.zip -k 21 --skipmer-m2n3 \
-o l_amylovorus.pang.skipm2n3.zip --scaled 50 --no-dna 

sourmash scripts pangenome_ranktable l_amylovorus.pang.prot.10.zip -o l_amylovorus.pang.prot.10.csv \
-k 15 --scaled 10 --protein --no-dna


In [ ]:
srun --account=ctbrowngrp -p med2 -J roary -t 6:00:00 -c 10 --mem=50gb --pty bash

snakemake --use-conda --resources mem_mb=50000 --rerun-triggers mtime \
-c 10 --rerun-incomplete -k -s pangenome_refgenomes.smk -n

In [ ]:
# now for protein and transcripts?
sourmash sig cat *.zip --protein -o ../l_amylovorus.prot.zip
sourmash sig cat *.zip  -o ../l_amylovorus.fna.zip

# downsample? different ksizes?? (15,18,21)
snakemake --use-conda --resources mem_mb=50000 --rerun-triggers mtime \
-c 10 --rerun-incomplete -k -s pangenome_prot.smk -n

# downsample protein sig
sourmash sig downsample \
/group/ctbrowngrp2/amhorst/2025-pangenome/results/pangenome/l_amylovorus/sourmash/protein/l_amylovorus.prot.zip \
-k 21 --protein --scaled 1000 \
-o /group/ctbrowngrp2/amhorst/2025-pangenome/results/pangenome/l_amylovorus/sourmash/l_amylovorus.21.1000.prot.zip 

# pangenome merge prot sig
sourmash scripts pangenome_merge \
/group/ctbrowngrp2/amhorst/2025-pangenome/results/pangenome/l_amylovorus/sourmash/l_amylovorus.21.1000.prot.zip \
-k 21 -o /group/ctbrowngrp2/amhorst/2025-pangenome/results/pangenome/l_amylovorus/sourmash/l_amylovorus.pang.21.1000.prot.zip \
--scaled 1000 --protein --no-dna 


# pangenome ranktable
sourmash scripts pangenome_ranktable \
/group/ctbrowngrp2/amhorst/2025-pangenome/results/pangenome/l_amylovorus/sourmash/l_amylovorus.pang.21.1000.prot.zip \
-o /group/ctbrowngrp2/amhorst/2025-pangenome/results/pangenome/l_amylovorus/sourmash/l_amylovorus.rankt.21.1000.prot.csv \
-k 21 --scaled 1000 --protein --no-dna

In [ ]:
# Try for protein
sourmash sig cat *.zip --protein -o h_porci.prot.zip

sourmash scripts pangenome_merge h_porci.prot.zip \
-k 15 --scaled 100 -o h_porci.pang.prot.zip --protein --no-dna


sourmash scripts pangenome_merge h_porci.gtdb.zip -o h_porci.pang.21.1000.zip -k 21 --scaled 1000


sourmash sig downsample h_porci.gtdb.zip -k 21 --scaled 100 -o h_porci.100.zip

sourmash scripts pangenome_merge h_porci.100.zip -o h_porci.pang.21.100.zip -k 21 --scaled 100


In [ ]:
        sourmash scripts pangenome_merge {input.sig_gtdb} -k {wildcards.ksize} \
        -o {output.pang} --scaled {wildcards.scaled} && \
        sourmash scripts pangenome_ranktable {output.pang} -o {output.rankt} \
        -k {wildcards.ksize} --scaled {wildcards.scaled}